In [52]:
import random
import numpy as np
from data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

cifar10_dir = 'datasets/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [53]:
# Preprocessing: reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))


# As a sanity check, print out the shapes of the data
print ('Training data shape: ', X_train.shape)
print ('Test data shape: ', X_test.shape)


Training data shape:  (50000, 3072)
Test data shape:  (10000, 3072)


In [54]:
mean_image = np.mean(X_train, axis=0)

In [55]:
# second: subtract the mean image from train and test data
X_train -= mean_image
X_test -= mean_image

In [56]:
input_size = 32 * 32 * 3
hidden_size = 50
num_classes = 10

In [57]:

np.random.seed(0)
feats = {}
feats['Weight1'] = np.random.randn(input_size, hidden_size)*0.00001
feats['bias1'] = np.zeros(hidden_size)
feats['Weight2'] = np.random.randn(hidden_size, num_classes)*0.00001
feats['bias2'] = np.zeros(num_classes)

In [58]:
def loss_calc(features, X, y, reg):
    W1, b1, W2, b2 = features['Weight1'],features['bias1'], features['Weight2'], features['bias2']
    z1 = X.dot(W1) + b1 
    loss=0.0
    relu = np.maximum(0, z1) # pass through ReLU activation function
    scores = relu.dot(W2) + b2
    expscore = np.exp(scores)
    sum = np.sum(expscore, axis=1, keepdims=True)
    p = expscore/sum
    prob = -np.log(p[np.arange(X.shape[0]), y])
    loss += np.sum(prob) / X.shape[0]
    regularization_loss = 0.5 * reg * np.sum(W1 * W1) + 0.5*reg*np.sum(W2*W2)
    loss += regularization_loss
    dscores = p
    N = X.shape[0]
    dscores[range(N),y] -= 1
    dscores /= N
    grads = {}
    # W2 and b2
    grads['W2'] = np.dot(relu.T, dscores)
    grads['b2'] = np.sum(dscores, axis=0)
    # next backprop into hidden layer
    dhidden = np.dot(dscores, W2.T)
    # backprop the ReLU non-linearity
    dhidden[relu <= 0] = 0
    # finally into W,b
    grads['W1'] = np.dot(X.T, dhidden)
    grads['b1'] = np.sum(dhidden, axis=0)

    # add regularization gradient contribution
    grads['W2'] += reg * W2
    grads['W1'] += reg * W1
    return loss, grads


In [59]:
def train(features, Xtr, ytr, num_iters, batch_size, learning_rate, reg):
    num_train = Xtr.shape[0]
    iterations = max(num_train / batch_size, 1)

    loss_history = []
    train_acc_history = []
    val_acc_history = []

    for it in xrange(num_iters):
      index = np.random.choice(np.arange(num_train), batch_size)
      X_batch = Xtr[index]
      y_batch = ytr[index]
      loss, grads = loss_calc(features, X_batch, y_batch, reg)
      features['Weight1'] += -learning_rate * grads['W1']
      features['bias1'] += -learning_rate * grads['b1']
      features['Weight2'] += -learning_rate * grads['W2']
      features['bias2'] += -learning_rate * grads['b2']
    return features


In [65]:
learnt_feats = train(feats, X_train, y_train,
            num_iters=10000, batch_size=200,
            learning_rate=1e-4, reg=0.5)

In [66]:
def predict(features, X):
    z1 = X.dot(features['Weight1']) + features['bias1']
    relu = np.maximum(0, z1) # pass through ReLU activation function
    scores = relu.dot(features['Weight2']) + features['bias2']
    y_pred = np.argmax(scores, axis=1)
    return y_pred

In [67]:
y_pred = predict(learnt_feats, X_test)

In [71]:
print (y_pred)

[3 8 8 ..., 3 5 7]


In [73]:
print ("The accuracy of the model is ",np.sum(y_pred == y_test)*100.0/y_test.shape[0])

The accuracy of the model is  48.18
